<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Denmark_Copenhagen_Meteorological_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
base_url = "https://opendataapi.dmi.dk/v2/metObs/collections/observation/items"
start_date = "2024-01-01T00:00:00Z"
end_date = "2025-01-01T00:00:00Z"
limit = 300000

# Define the specific tasks as requested
tasks = [
    # 1. AIRPORT (06180) for Wind and Temp
    {"station": "06180", "param": "temp_dry",   "name": "Temperature_C"},
    {"station": "06180", "param": "wind_speed", "name": "Wind_Speed_ms"},

    # 2. TOLDBOD (06187) for Radiation
    # We try 'radia_glob_past1h' first (Hourly Average), as it's standard for DMI stats
    {"station": "06187", "param": "radia_glob_past1h", "name": "Global_Radiation_Wm2"},
]

dfs = []

print(f"Fetching data for Copenhagen (Toldbod & Airport)...")

for task in tasks:
    print(f"   > Requesting {task['name']} from Station {task['station']}...")

    params = {
        "stationId": task['station'],
        "datetime": f"{start_date}/{end_date}",
        "parameterId": task['param'],
        "limit": limit
    }

    try:
        r = requests.get(base_url, params=params)

        # If the 'past1h' parameter fails for radiation, try the instant one
        if r.status_code != 200 and task['name'] == "Global_Radiation_Wm2":
            print("     'past1h' not found, trying instant 'radia_glob'...")
            params['parameterId'] = 'radia_glob'
            r = requests.get(base_url, params=params)

        if r.status_code == 200:
            data = r.json().get('features', [])
            print(f"     Received {len(data)} observations.")

            records = []
            for item in data:
                records.append({
                    'Time': item['properties']['observed'],
                    task['name']: item['properties']['value']
                })

            df_temp = pd.DataFrame(records)
            if not df_temp.empty:
                df_temp['Time'] = pd.to_datetime(df_temp['Time'])
                # Resample 10-min data to Hourly Averages
                df_temp = df_temp.set_index('Time').resample('h').mean()
                dfs.append(df_temp)
            else:
                print(f"     WARNING: Zero rows found for {task['name']} at {task['station']}.")
        else:
            print(f"     Error {r.status_code}: {r.text}")

    except Exception as e:
        print(f"     Connection failed: {e}")

    time.sleep(1)

# --- MERGE & SAVE ---
if dfs:
    print("Merging datasets...")
    df_final = pd.concat(dfs, axis=1)

    # Sort
    df_final = df_final.sort_index()

    # Interpolate small gaps
    df_final = df_final.interpolate(method='linear', limit=2)

    # Clean Radiation (NaN -> 0 for night, remove negatives)
    if 'Global_Radiation_Wm2' in df_final.columns:
        df_final['Global_Radiation_Wm2'] = df_final['Global_Radiation_Wm2'].fillna(0)
        df_final.loc[df_final['Global_Radiation_Wm2'] < 0, 'Global_Radiation_Wm2'] = 0

    # Drop empty rows
    df_final = df_final.dropna(how='all')

    print(df_final.head())

    filename = "Copenhagen_Toldbod_Airport_2024.csv"
    df_final.to_csv(filename)
    print(f"\nSuccess! Saved to: {filename}")
else:
    print("Failed to retrieve data.")

Fetching data for Copenhagen (Toldbod & Airport)...
   > Requesting Temperature_C from Station 06180...
     Received 52671 observations.
   > Requesting Wind_Speed_ms from Station 06180...
     Received 52671 observations.
   > Requesting Global_Radiation_Wm2 from Station 06187...
     Received 8777 observations.
Merging datasets...
                           Temperature_C  Wind_Speed_ms  Global_Radiation_Wm2
Time                                                                         
2024-01-01 00:00:00+00:00       5.433333       5.233333                   0.0
2024-01-01 01:00:00+00:00       5.466667       4.183333                   0.0
2024-01-01 02:00:00+00:00       5.483333       5.383333                   0.0
2024-01-01 03:00:00+00:00       5.450000       5.400000                   0.0
2024-01-01 04:00:00+00:00       5.650000       4.600000                   0.0

Success! Saved to: Copenhagen_Toldbod_Airport_2024.csv
